System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65e (2023-01-08 06:45 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 16 × AMD Ryzen 7 5800 8-Core Processor              
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, znver3)
  Threads: 1 on 16 virtual cores


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw3`


Status `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw3\Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [31c24e10] Distributions v0.25.88
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random


In [3]:
using LinearAlgebra, Random
using BenchmarkTools, Distributions

Consider a linear mixed effects model
$$
    \mathbf{Y}_i = \mathbf{X}_i \boldsymbol{\beta} + \mathbf{Z}_i \boldsymbol{\gamma} + \boldsymbol{\epsilon}_i, \quad i=1,\ldots,n,
$$
where   
- $\mathbf{Y}_i \in \mathbb{R}^{n_i}$ is the response vector of $i$-th individual,  
- $\mathbf{X}_i \in \mathbb{R}^{n_i \times p}$ is the fixed effect predictor matrix of $i$-th individual,  
- $\mathbf{Z}_i \in \mathbb{R}^{n_i \times q}$ is the random effect predictor matrix of $i$-th individual,  
- $\boldsymbol{\epsilon}_i \in \mathbb{R}^{n_i}$ are multivariate normal $N(\mathbf{0}_{n_i},\sigma^2 \mathbf{I}_{n_i})$,  
- $\boldsymbol{\beta} \in \mathbb{R}^p$ are fixed effects, and  
- $\boldsymbol{\gamma} \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \boldsymbol{\Sigma}_{q \times q}$) independent of $\boldsymbol{\epsilon}_i$.

## Q1 Formula (10 pts)

Write down the log-likelihood of the $i$-th datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$ given parameters $(\boldsymbol{\beta}, \boldsymbol{\Sigma}, \sigma^2)$. 

**Sol.**

In linear mixed effects model, we know the $\mathbf{Y}_i$ follows multivariate normal distribution:

$$
\mathbf{Y}_i \sim N_{n_i}(\mathbf{X}_i \boldsymbol{\beta}, \mathbf{V}_i)
$$

in which $\mathbf{V}_i=\mathbf{Z}_i \boldsymbol{\Sigma} \mathbf{Z}_i^T + \sigma^2 \mathbf{I}_{n_i}$ is the covariance matrix.

Then we can get the pdf for $\mathbf{Y}_i$:

$$
f(\mathbf{y}_i|\boldsymbol{\beta},\boldsymbol{\Sigma},\sigma^2) = \frac{1}{(2\pi)^{n_i/2} |\mathbf{V}_i|^{1/2}} \exp\left(-\frac{1}{2}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \mathbf{V}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})\right),
$$

Now, we can write the log-likelihood of $\mathbf{Y}_i$:

$$
\ell_i(\boldsymbol{\beta},\boldsymbol{\Sigma},\sigma^2) = -\frac{n_i}{2} \log(2\pi) -\frac{1}{2} \log |\mathbf{V}_i| -\frac{1}{2}(\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta})^T \mathbf{V}_i^{-1} (\mathbf{y}_i - \mathbf{X}_i \boldsymbol{\beta}).
$$

## Q2 Start-up code

Use the following template to define a type `LmmObs` that holds an LMM datum $(\mathbf{y}_i, \mathbf{X}_i, \mathbf{Z}_i)$. 

In [4]:
# define a type that holds LMM datum
struct LmmObs{T <: AbstractFloat}
    # data
    y :: Vector{T}
    X :: Matrix{T}
    Z :: Matrix{T}
    # working arrays
    # whatever intermediate vectors/arrays you may want to pre-allocate
    storage_p  :: Vector{T}
    storage_q  :: Vector{T}
    xtx        :: Matrix{T}
    ztx        :: Matrix{T}
    ztz        :: Matrix{T}
    zty        :: Vector{T}
    xty        :: Vector{T}
    ztxB       :: Vector{T}
    yty        :: T
    storage_qq :: Matrix{T}
end

# constructor
function LmmObs(
        y::Vector{T}, 
        X::Matrix{T}, 
        Z::Matrix{T}
        ) where T <: AbstractFloat
    storage_p  = Vector{T}(undef, size(X, 2))
    storage_q  = Vector{T}(undef, size(Z, 2))
    xtx        = transpose(X) * X
    ztx        = transpose(Z) * X
    ztz        = transpose(Z) * Z
    zty        = transpose(Z) * y
    xty        = transpose(X) * y
    ztxB       = similar(zty)
    yty        = transpose(y) * y
    storage_qq = similar(ztz)
    
    LmmObs(y, X, Z, storage_p, storage_q, xtx, ztx, ztz, zty, xty, ztxB, yty, storage_qq)
end

LmmObs

Write a function, with interface   
```julia
logl!(obs, β, L, σ²)
```
that evaluates the log-likelihood of the $i$-th datum. Here `L` is the lower triangular Cholesky factor from the Cholesky decomposition `Σ=LL'`. Make your code efficient in the $n_i \gg q$ case. Think the intensive longitudinal measurement setting.  

In [5]:
function logl!(
        obs :: LmmObs{T}, 
        β   :: Vector{T}, 
        L   :: Matrix{T},  
        σ²  :: T) where T <: AbstractFloat
    n, p, q = size(obs.X, 1), 0.0, size(obs.Z, 2)      
    L .= LowerTriangular(L)
    
    obs.storage_qq .= obs.ztz
    BLAS.syrk!('U', 'T', 1.0, L, 1/σ², obs.storage_qq)
    obs.storage_qq .= Symmetric(obs.storage_qq)
    LAPACK.potrf!('L', obs.storage_qq)
    obs.storage_qq .= LowerTriangular(obs.storage_qq)
    
    obs.storage_q .= obs.zty
    BLAS.gemv!('N', -1.0, obs.ztx, β, 1.0, obs.storage_q)
    BLAS.trsv!('L', 'N', 'N', obs.storage_qq, obs.storage_q)
    
    BLAS.gemv!('N', 1.0, obs.xtx, β, 0.0, obs.storage_p)
    
    for i in 1:q
        p -= log(obs.storage_qq[i, i])
        p += log(L[i, i])
    end

    n = - (n//2) * log(2π * σ²) + p - 
        (1//2) * (1 / σ²) * (obs.yty - 2 * dot(obs.xty, β) + dot(β, obs.storage_p)) +
        (1//2) * ((1 / σ²) ^ 2) * dot(obs.storage_q, obs.storage_q)
    n
end

logl! (generic function with 1 method)

## Basic idea

$$
\begin{aligned}
l\left(\beta, \Sigma, \sigma^{2}\right)
& =-\frac{n}{2} \log (2 \pi)-\frac{1}{2} \log |V|-\frac{1}{2}(Y-X \beta)^{\prime} V^{-1}(Y-X \beta) \\
& =-\frac{n}{2} \log (2 \pi)-\frac{1}{2} \log \left|z \sum z^{\prime}+\sigma^{2} I_{n}\right|-\frac{1}{2}\left(Y-X(\beta)^{\prime}\left(z \sum z^{\prime}+\sigma^{2} I_{n}\right)^{-1}(Y-x \beta)\right)\\
& =-\frac{n}{2} \log (2 \pi)-\frac{n}{2} \log  \sigma^{2}-\frac{1}{2} \log \left|I_{3}+z^{\prime} z \Sigma / \sigma^{2}\right|-\frac{1}{2 \sigma^{2}}(Y-x \beta)^{\prime}(Y-x \beta)+\frac{1}{2 \sigma^{2}}(Y-x \beta)^{\prime} z\left(\Sigma^{-1}+z^{\prime} z / \sigma^{2}\right)^{-1} z^{\prime}(Y-x \beta) \\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log \left|I_{3}+z^{\prime} z \Sigma / \sigma^{2}\right|-\frac{1}{2 \sigma^{2}}(Y-x \beta)^{\prime}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left(z^{\prime} Y-z^{\prime}x \beta\right)^{\prime}\left(\Sigma^{-1}+z^{\prime} z /\sigma^{2}\right)^{-1}\left(z^{\prime}Y-z^{\prime}x \beta\right)\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log \left|I_{3}+z^{\prime} z \Sigma / \sigma^{2}\right|-\frac{1}{2 \sigma^{2}}(Y-x \beta)^{\prime}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left(z^{\prime} Y-z^{\prime}x \beta\right)^{\prime}\left(L^{\prime}L\right)^{-1}\left(z^{\prime}Y-z^{\prime}x \beta\right)\\
& =-\frac{n}{2} \log (2 \pi\sigma^{2}) -\frac{1}{2} \log \left|I_{3}+z^{\prime} z \Sigma / \sigma^{2}\right|-\frac{1}{2 \sigma^{2}}(Y-x \beta)^{\prime}(Y-x \beta)+\frac{1}{2 \sigma^{4}}\left|L^{-1}\left(z^{\prime} Y-z^{\prime}x \beta\right)\right|^2_2
\end{aligned}
$$

**Hint**: This function shouldn't be very long. Mine, obeying 92-character rule, is 30 lines. If you find yourself writing very long code, you're on the wrong track. Think about algorithm (flop count) first then use BLAS functions to reduce memory allocations.

## Q3 Correctness (15 pts)

Compare your result (both accuracy and timing) to the [Distributions.jl](https://juliastats.org/Distributions.jl/stable/multivariate/#Distributions.AbstractMvNormal) package using following data.

In [6]:
Random.seed!(257)
# dimension
n, p, q = 2000, 5, 3
# predictors
X  = [ones(n) randn(n, p - 1)]
Z  = [ones(n) randn(n, q - 1)]
# parameter values
β  = [2.0; -1.0; rand(p - 2)]
σ² = 1.5
Σ  = fill(0.1, q, q) + 0.9I
# generate y
y  = X * β + Z * rand(MvNormal(Σ)) + sqrt(σ²) * randn(n)

# form an LmmObs object
obs = LmmObs(y, X, Z)

LmmObs{Float64}([-1.450910909560209, 1.5185224894450862, 5.265021705624027, 4.485272594164557, 0.6949699666429332, 1.7723256696372407, 1.1065838446466518, 3.7291668118296073, 4.288899999400642, 2.8241842645202406  …  4.058027151891635, 1.0909724390970443, 0.026692243086209766, -0.8927757653299448, 6.94725248926293, 3.519302085567343, 4.914007299083773, 2.1610206566690797, 1.857389542694909, 6.513818951020866], [1.0 0.6790633442371218 … 0.5400611947971554 -0.632040682052606; 1.0 1.2456776800889142 … -0.4818455756130373 0.6467830314674976; … ; 1.0 0.0733124748775436 … 0.6125080259511859 0.4181258283983667; 1.0 -1.336609049786048 … -0.18567490803712938 1.0745977099307227], [1.0 -1.0193326822839996 -0.15855601254314888; 1.0 1.7462667837699666 -0.4584376230657152; … ; 1.0 1.4843185594903878 0.42458303115266854; 1.0 0.3791714762820068 0.25150666970865837], [7.9258207078e-312, 7.9258207078e-312, 7.9258207078e-312, 7.9258207078e-312, 7.926028663467e-312], [6.95290918929967e-310, 7.92582079337e

This is the standard way to evaluate log-density of a multivariate normal, using the Distributions.jl package. Let's evaluate the log-likelihood of this datum.

In [7]:
μ  = X * β
Ω  = Z * Σ * transpose(Z) +  σ² * I
mvn = MvNormal(μ, Symmetric(Ω)) # MVN(μ, Σ)
logpdf(mvn, y)

-3256.179335805831

Check that your answer matches that from Distributions.jl

In [8]:
L = Matrix(cholesky(Σ).L)
L = inv(L)
logl!(obs, β, L, σ²)

-3256.1793358058258

**You will lose all 15 + 30 + 30 = 75 points** if the following statement throws `AssertionError`.

In [9]:
@assert logl!(obs, β, L, σ²) ≈ logpdf(mvn, y)

## Q4 Efficiency (30 pts)

Benchmarking your code and compare to the Distributions.jl function `logpdf`.

In [10]:
# benchmark the `logpdf` function in Distribution.jl
bm1 = @benchmark logpdf($mvn, $y)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  250.700 μs …   8.284 ms  ┊ GC (min … max): 0.00% … 93.12%
 Time  (median):     295.600 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   308.081 μs ± 120.381 μs  ┊ GC (mean ± σ):  0.50% ±  1.32%

       ▂▄▆█▆▆▄▄▃▄▂▂                                              
  ▁▂▂▄▇█████████████▇▇▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  251 μs           Histogram: frequency by time          459 μs <

 Memory estimate: 31.52 KiB, allocs estimate: 3.

In [11]:
# benchmark your implementation
bm2 = @benchmark logl!($obs, $β, $L, $σ²)

BenchmarkTools.Trial: 10000 samples with 190 evaluations.
 Range (min … max):  525.263 ns …  2.337 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     541.053 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   551.010 ns ± 58.629 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▆█▇▅▃▁                                                      ▂
  ████████▇▇▆▄▅▄▅▆▅▅▆▆▆▆▅▄▆▇▆▅▄▅▄▅▄▅▅▄▃▄▄▅▄▃▁▃▄▄▄▃▅▃▄▄▄▄▄▄▅▇▇▇ █
  525 ns        Histogram: log(frequency) by time       866 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

The points you will get is
$$
\frac{x}{1000} \times 30,
$$
where $x$ is the speedup of your program against the standard method.

In [12]:
# this is the points you'll get
clamp(median(bm1).time / median(bm2).time / 1000 * 30, 0, 30)

16.390272373540856

**Hint**: Apparently I am using 1000 as denominator because I expect your code to be at least $1000 \times$ faster than the standard method.

## Q5 Memory (30 pts)

You want to avoid memory allocation in the "hot" function `logl!`. You will lose 1 point for each `1 KiB = 1024 bytes` memory allocation. In other words, the points you get for this question is

In [13]:
clamp(30 - median(bm2).memory / 1024, 0, 30)

30.0

**Hint**: I am able to reduce the memory allocation to 0 bytes.

## Q6 Misc (15 pts)

Coding style, Git workflow, etc. For reproducibity, make sure we (TA and myself) can run your Jupyter Notebook. That is how we grade Q4 and Q5. If we cannot run it, you will get zero points.